In [144]:
# -*- coding: UTF-8 -*-

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
import re

from docxtpl import DocxTemplate
from docxcompose.composer import Composer
from docx import Document
from docx2pdf import convert

In [6]:
df = pd.read_excel('data/probs.xlsx',dtype=str)
name_file_template_doc = 'data/Шаблон профессиональной пробы 18.06'

In [7]:
df

,ID,Время создания,Профессиональная среда,Наименование профессионального направления,ФИО автора программы,Должность/звание автора программы,Регион,Город,Электронная почта автора,Контактный телефон,...,Постановка цели и задачи в рамках пробы.,"Демонстрация итогового результата, продукта.",Подробная инструкция по выполнению задания.,Рекомендации для наставника по организации процесса выполнения задания.,Критерии успешного выполнения задания.,"Рекомендации для наставника по контролю результата, процедуре оценки.",Вопросы для рефлексии обучающихся.,Введите инфраструктурный лист так как указано выше,"Введите дополнительные источники литературы, ссылки на полезную информацию, ссылки на видео и т.п. \nКаждый пункт должен заканчиваться точкой с запятой","Приложите дополнительные файлы (фотографии, чертежи, схемы и т.п.) если они используются в профпробе."
0,1416819968,2023-06-19 11:06:57,техническая,Программист,Иванов Иван Иванович,CDO ЦОПП Бурятия,Забайкальский край,г. Чита,test@test.ru,89503814183,...,"Даже промежуточные результаты, полученные учас...","Даже промежуточные результаты, полученные учас...",- рассказ наставника о конкретной профессионал...,- рассказ наставника о конкретной профессионал...,Площадками по реализации практических занятий ...,Площадками по реализации практических занятий ...,Площадками по реализации практических занятий ...,"Бумага А4;белая,плотность 80г/м²;20 листов;20/...",NaN,NaN
1,1416590916,2023-06-19 04:54:47,Умная,Педагог-психолог,Белькова Мария Родионовна,методист ЦОПП РБ,Республика Бурятия,гор. Улан-Удэ,belykova85@mail.ru,89021626856,...,"Задача школьников состоит в том, чтобы помочь ...",Участники профпробы должны - разработать план ...,"Шаг 1: Введение. Объяснить школьнику, что это ...","При выполнении задания, наставник может предла...",1. Анализ проблемы: Школьник должен продемонст...,Оценка выполнения задания должна основываться ...,Школьникам необходимо закончить фразы: «Я увид...,Комплект мультимедийного оборудования (проекто...,•\tАфанасьев А. Тайм-менеджмент для детей. Меч...,NaN


In [8]:
df.fillna('',inplace=True)

In [9]:
df.columns

Index(['ID', 'Время создания', 'Профессиональная среда',
       'Наименование профессионального направления', 'ФИО автора программы',
       'Должность/звание автора программы', 'Регион', 'Город',
       'Электронная почта автора', 'Контактный телефон',
       'Уровень сложности программы', 'Формат проведения', 'Время проведения',
       'Возрастная категория участников / 6-7 класс',
       'Возрастная категория участников / 8-9 класс',
       'Возрастная категория участников / 10-11 класс',
       'Допустимая нозологическая группа/группы / Нарушение зрения (Слепые, Слабовидяшие)',
       'Допустимая нозологическая группа/группы / Нарушение слуха (Глухие, Слабослышащие)',
       'Допустимая нозологическая группа/группы / Нарушение опорно-двигательного аппарата (НОДА)',
       'Допустимая нозологическая группа/группы / Общие заболевания (нарушение дыхательной системы, пищеварительной, эндокринной систем, сердечно-сосудистой системы и т.д.)',
       'Допустимая нозологическая группа/груп

In [10]:
df.loc[:,'Допустимая нозологическая группа/группы / Нарушение зрения (Слепые, Слабовидяшие)':'Допустимая нозологическая группа/группы / Нарушение интеллекта (умственная отсталость)']

,"Допустимая нозологическая группа/группы / Нарушение зрения (Слепые, Слабовидяшие)","Допустимая нозологическая группа/группы / Нарушение слуха (Глухие, Слабослышащие)",Допустимая нозологическая группа/группы / Нарушение опорно-двигательного аппарата (НОДА),"Допустимая нозологическая группа/группы / Общие заболевания (нарушение дыхательной системы, пищеварительной, эндокринной систем, сердечно-сосудистой системы и т.д.)",Допустимая нозологическая группа/группы / Тяжёлые нарушение речи,Допустимая нозологическая группа/группы / Нарушение интеллекта (умственная отсталость)
0,"Нарушение зрения (Слепые, Слабовидяшие)",,,Общие заболевания (нарушение дыхательной систе...,Тяжёлые нарушение речи,
1,,,Нарушение опорно-двигательного аппарата (НОДА),,,


In [11]:
df.loc[:,'Допустимая нозологическая группа/группы / Нарушение зрения (Слепые, Слабовидяшие)':'Допустимая нозологическая группа/группы / Нарушение интеллекта (умственная отсталость)'].sum(axis=1)

0    Нарушение зрения (Слепые, Слабовидяшие)Общие з...
1       Нарушение опорно-двигательного аппарата (НОДА)
dtype: object

In [150]:
lst_name_columns = ['ID','Время_создания','Профессиональная_проба','Наименование_профессионального_направления','Автор_программы','Контакты_автора',
                    'Уровень_сложности','Формат_проведения','Время_проведения','Возрастная_категория','Допустимые_нозологии',
                    'Спец_условия','Возможность_проведения','Краткое_описание','Перспективы','Навыки_знания','Интересные_факты',
                    'Связь_пробы','Постановка_цели','Демонстрация','Инструкция','Рекомендации_организация','Критерии','Рекомендации_контроль',
                    'Рефлексия','Инфраструктурный_лист','Доп_источники','Доп_файлы']

In [151]:
df.columns = lst_name_columns

In [153]:
 # Заполняем Nan
df.fillna(' ', inplace=True)



In [154]:
#df = df.applymap(lambda x:x.replace("\n", " ")) # удаляем перенос строк
#df = df.applymap(lambda x:x.replace("\r", "")) # удаляем возврат каретки
df = df.applymap(lambda x:x.replace("\u00A0", " ")) # удаляем символ неразрывного пробела

In [155]:
df

,ID,Время_создания,Профессиональная_проба,Наименование_профессионального_направления,Автор_программы,Контакты_автора,Уровень_сложности,Формат_проведения,Время_проведения,Возрастная_категория,...,Постановка_цели,Демонстрация,Инструкция,Рекомендации_организация,Критерии,Рекомендации_контроль,Рефлексия,Инфраструктурный_лист,Доп_источники,Доп_файлы
0,1413830697,2023-06-15 10:11:47,умная,Педагог-психолог,"Мария Родионовна Белькова, методист центра опе...","Республика Бурятия, город Улан-Удэ, belykova85...",базовый,очный,90 мин,"8-9 класс,10-11 класс",...,"Задача школьников состоит в том, чтобы помочь ...",Участники профпробы должны - разработать план ...,"Шаг 1: Введение\nОбъяснить школьнику, что это ...","При выполнении задания, наставник может предла...",1. Анализ проблемы: Школьник должен продемонст...,Оценка выполнения задания должна основываться ...,Школьникам необходимо закончить фразы: «Я увид...,"Бумага А4;белая,плотность 80г/м²;20 листов;20/...",•\tАфанасьев А. Тайм-менеджмент для детей. Меч...,https://forms.yandex.ru/cloud/files?path=%2F68...
1,1413835059,2023-06-15 10:17:13,тестовая,Тестировщик,"Иванов Иван Иванович, методист ЦОПП Бурятия.","Республика Бурятия, город Улан-Удэ, ivanov@tes...",ознакомительный,онлайн,30 минут,10-11 класс,...,"Для ознакомительного уровня, максимальное коли...",Площадками по реализации практических занятий ...,Итоговая оценка работы участника в рамках прог...,"В материалах, используемых для оценки результа...",без игровой адаптации. Позитивным элементом мо...,- рассказ наставника о конкретной профессионал...,Площадками по реализации практических занятий ...,"Ноутбук;Full HD (1920x1080), IPS, Intel Celero...",,


In [156]:
 # Конвертируем датафрейм в список словарей
data = df.to_dict('records')

In [157]:
for idx,row in enumerate(data):
    doc = DocxTemplate(name_file_template_doc)
    context = row
    # print(context)
    doc.render(context)
    doc.save(f'data/{idx}.docx')